In [1]:
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as transforms

import math

from IPython.display import Audio
import matplotlib.pyplot as plt
import librosa
import os
from torchaudio.utils import download_asset

from scipy.io import wavfile
import math
import numpy as np
from scipy import signal
from pathlib import Path
import scipy.signal as sps
from scipy.signal import butter, lfilter
import random
import pydub
import uuid
from pydub import AudioSegment, effects
import soundfile as sf
import shutil
import openpyxl

import xlsxwriter

print(torch.__version__)
print(torchaudio.__version__)

c:\Users\Edward\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


2.0.1+cu117
2.0.2+cu117


Step 1) Creating files for all data and classifications with its respective excel page

In [2]:
# All locations
path_clean = "dataset_all/clean/"
path_noisy = "dataset_all/noisy/"
path_noisy_3dB = "dataset_all/-3dB/"
path_noisy_6dB = "dataset_all/-6dB/"
path_noisy_9dB = "dataset_all/-9dB/"
path = "dataset_all"
path2 = "dataset_classification"
path3 = path + "/all_info.xlsx"
path4 = path2 + "/classification_info.xlsx"

In [4]:
import os
import xlsxwriter

isExist = os.path.exists(path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(path)
   os.makedirs(path + "/clean")
   os.makedirs(path + "/noisy")
   os.makedirs(path + "/-3dB")
   os.makedirs(path + "/-6dB")
   os.makedirs(path + "/-9dB")
   print("the %s directory is created!", path)

isExist2 = os.path.exists(path2)
if not isExist2:
   # Create a new directory because it does not exist
   os.makedirs(path2)
   os.makedirs(path2 + "/clean")
   os.makedirs(path2 + "/noisy")
   print("The %s directory is created!", path2)

isExist3 = os.path.exists(path3)
if not isExist3:
   workbook1 = xlsxwriter.Workbook(path3)
   worksheet1 = workbook1.add_worksheet()
   worksheet1.write(0, 0, "Sample Number")
   worksheet1.write(0, 1, "Name")
   worksheet1.write(0, 2, "Length(s)")
   worksheet1.write(0, 3, "Type of Speech")
   worksheet1.write(0, 4, "SNR(dB)")
   worksheet1.write(0, 5, "Noise Type")
   worksheet1.write(0, 6, "Noise Channel")
   worksheet1.write(0, 7, "Speaker ID")
   worksheet1.write(0, 8, "Passage ID")
   print("The excel file 1 is created!")
   workbook1.close()

isExist4 = os.path.exists(path4)
if not isExist4:
   workbook2 = xlsxwriter.Workbook(path4)
   worksheet2 = workbook2.add_worksheet()
   worksheet2.write(0, 0, "Sample Number")
   worksheet2.write(0, 1, "Name")
   worksheet2.write(0, 2, "Length(s)")
   worksheet2.write(0, 3, "Type of Speech")
   worksheet2.write(0, 4, "SNR(dB)")
   worksheet2.write(0, 5, "Noise Type")
   worksheet2.write(0, 6, "Noise Channel")
   worksheet2.write(0, 7, "Speaker ID")
   worksheet2.write(0, 8, "Passage ID")
   print("The excel file 2 is created!")
   workbook2.close()


Step 2) Locations of speech and noise

In [5]:
speech_fpath = "speech/VCTK-Corpus/wav48/"
speech_file_entries = os.listdir(speech_fpath)
print(len(speech_file_entries))

109


In [6]:
noise_fpath = "noise/noise/"
noise_file_entries = os.listdir(noise_fpath)
print(len(noise_file_entries))

17


Step 3) Normalise and downsample speech from 48K to 16K and normalise noise

In [7]:
def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [8]:
def downsampling(clip):
    samplerate, data = wavfile.read(clip)
    Fs1 = samplerate
    Fs2 = 16000
    N = len(data)
    total_time = (N-1)/Fs1
    Max_Signal_Frequency =Fs2/2
    New_sample_amount = math.ceil(Fs2*total_time)
    Single_Channel = np.zeros(New_sample_amount)
    bit_depth = data.getsampwidth() * 8
    data = data/(2**(bit_depth-1))
    Original_signal = data
    Anti_Aliased_signal = np.array(butter_lowpass_filter(Original_signal,Max_Signal_Frequency,Fs1))
    Down_sampled_signal = np.array(sps.resample(Anti_Aliased_signal,New_sample_amount))
    Single_Channel = Down_sampled_signal
    Transformed_single_channel = Single_Channel.transpose()
    return Transformed_single_channel, Fs2

In [9]:
def normalise(inclip, outclip):
    rawsound = AudioSegment.from_wav(inclip)  
    normalizedsound = effects.normalize(rawsound)  
    normalizedsound.export(outclip, format = 'wav')

Step 4) Adding noise to speech

In [10]:
def adding_noise_to_speech(speech, noise, i):
    speech, _ = torchaudio.load(speech)
    noise, _ = torchaudio.load(noise)

    # From a random point in the noise waveform make the size of the noise the same as the speech
    first = random.randint(0, noise.shape[1] - speech.shape[1])
    noise = noise[:, first:first + speech.shape[1]]

    # At all SNR levels add the noise to the speech
    snr_dbs = torch.tensor([-3, -6, -9])
    noisy_speeches = F.add_noise(speech, noise, snr_dbs)

    snr_db, noisy_speech = snr_dbs[0], noisy_speeches[0:1]
    torchaudio.save(path_noisy_3dB + "speech_" + str(i) + ".wav", noisy_speech, 16000, encoding="PCM_S", bits_per_sample=16)

    snr_db, noisy_speech = snr_dbs[1], noisy_speeches[1:2]
    torchaudio.save(path_noisy_6dB + "speech_" + str(i) + ".wav", noisy_speech, 16000, encoding="PCM_S", bits_per_sample=16)

    snr_db, noisy_speech = snr_dbs[2], noisy_speeches[2:3]
    torchaudio.save(path_noisy_9dB + "speech_" + str(i) + ".wav", noisy_speech, 16000, encoding="PCM_S", bits_per_sample=16)

Step 5) Create all and classification data with excel information

In [11]:
n = 10000

for i in range (0, n):

    #Selecting the clean speech

    # Random number to select the clean speech file (speaker) used
    ran_1 = random.randrange(0, len(speech_file_entries) - 2) # ignoring the last file used for testing
    speech_file = speech_file_entries[ran_1]
    speech_enter = speech_fpath + speech_file + "/"
    speech_entries = os.listdir(speech_enter)

    # Random number to select the passage used
    ran_2 = random.randrange(0, len(speech_entries) - 1)
    select_speech = speech_enter + speech_entries[ran_2]

    # Selecting the noise
    ran_3 = random.randrange(0, len(noise_file_entries) - 2)
    noise_file = noise_file_entries[ran_3]
    noise_enter = noise_fpath + noise_file + "/" + noise_file.split("_")[0] + "/"
    noise_entries = os.listdir(noise_enter)

    # Random number to select the channel used
    ran_4 = random.randrange(0, len(noise_entries) - 1)
    select_noise = noise_enter + noise_entries[ran_4]

    # downsampling then normalising the clean speech
    data, samplerate = downsampling(select_speech)
    sf.write(path_clean + "speech_" + str(i + 1) + ".wav", data, samplerate, 'PCM_16')
    normalise(path_clean + "speech_" + str(i + 1) + ".wav", path_clean + "speech_" + str(i + 1) + ".wav")
    
    # normalising the noisy speech
    normalise(select_noise, path_noisy + "noisy_" + str(i + 1) + ".wav")

    # Combinding the clean speech and the noise
    adding_noise_to_speech(path_clean + "speech_" + str(i + 1) + ".wav", path_noisy + "noisy_" + str(i + 1) + ".wav", i + 1)

    # Selecting the noisy speech for classification
    ran_5 = random.randint(1, 4)
    if ran_5 == 1:
        shutil.copy(path_clean + "speech_" + str(i + 1) + ".wav", path2 + "/clean")
        SNR = 0
        type_of_speech = "clean"
    elif ran_5 == 2:
        shutil.copy(path_noisy_3dB + "speech_" + str(i + 1) + ".wav", path2 + "/noisy")
        SNR = -3
        type_of_speech = "noisy"
    elif ran_5 == 3:
        shutil.copy(path_noisy_6dB + "speech_" + str(i + 1) + ".wav", path2 + "/noisy")
        SNR = -6
        type_of_speech = "noisy"
    else:
        shutil.copy(path_noisy_9dB + "speech_" + str(i + 1) + ".wav", path2 + "/noisy")
        SNR = -9
        type_of_speech = "noisy"


    # Adding the information to the excel file (path3/wf2 is classification, path4/wf1 is all)
    duration = round(librosa.get_duration(path=select_speech), 2) # Duration of speech
    passage_ID = speech_entries[ran_2].split('.')[0].split('_')[1] # Passage ID
    noise_channel = noise_entries[ran_4].split('.')[0]# Noise channel

    workfile1 = openpyxl.load_workbook(path3)
    workfile2 = openpyxl.load_workbook(path4)

    sheet1 = workfile1.active
    sheet2 = workfile2.active

    sheet1.cell(row=i+2, column=1).value = i+1
    sheet1.cell(row=i+2, column=2).value = "speech_" + str(i + 1) + ".wav"
    sheet1.cell(row=i+2, column=3).value = duration
    sheet1.cell(row=i+2, column=4).value = "noisy | clean"
    sheet1.cell(row=i+2, column=5).value = "0 | -3 | -6 | -9"
    sheet1.cell(row=i+2, column=6).value = noise_file
    sheet1.cell(row=i+2, column=7).value = noise_channel
    sheet1.cell(row=i+2, column=8).value = speech_file
    sheet1.cell(row=i+2, column=9).value = passage_ID

    sheet2.cell(row=i+2, column=1).value = i+1
    sheet2.cell(row=i+2, column=2).value = "speech_" + str(i + 1) + ".wav"
    sheet2.cell(row=i+2, column=3).value = duration
    sheet2.cell(row=i+2, column=4).value = type_of_speech
    sheet2.cell(row=i+2, column=5).value = SNR
    sheet2.cell(row=i+2, column=6).value = noise_file
    sheet2.cell(row=i+2, column=7).value = noise_channel
    sheet2.cell(row=i+2, column=8).value = speech_file
    sheet2.cell(row=i+2, column=9).value = passage_ID

    workfile1.save(path3)
    workfile2.save(path4)

    print("iteration: " + str(i + 1) + " noisy file: " + noise_file + " clean file: " + speech_file + " SNR: " + str(SNR) + " duration: " + str(duration) + " seconds", end = "\r")
